In [ ]:
# %run generic.ipynb

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
# COPY & PASTE from first.ipynb
# Generic Imports
from PIL import Image 
import scipy.io as sio 
# STD Imports
import glob # For searching files
import os # To remove file extension

# Constants
DATA_PATH: str = "./../data/WillowObject/WILLOW-ObjectClass/"

# 1. Gets Information
def getting(cat: Categories, n: int) -> list[tuple[Image.Image, np.array]]:
    images: list[str] = glob.glob(f"{DATA_PATH}{cat.value}/*.png")
    points: list[str] = glob.glob(f"{DATA_PATH}{cat.value}/*.mat")
    pairs = []
    set_points = set(points)

    for img_file in images:
        base, _ = os.path.splitext(img_file)
        mat_file = base + ".mat"
        if mat_file in set_points:
            img = Image.open(img_file)
            kps = np.array(sio.loadmat(mat_file)['pts_coord'])
            pairs.append((img, kps))
            if len(pairs) >= n:
                break

    return pairs

In [ ]:
# COPY & PASTE from first.ipynb
# Constants
RESIZE: int = 256
# 2. Resizes Values
def resizing(tup: tuple[Image.Image, np.array]) -> tuple[Image.Image, np.array]:
    img, kps = tup
    kps[0] *= RESIZE / img.size[0]
    kps[1] *= RESIZE / img.size[1]

    img = img.resize((RESIZE, RESIZE), resample=Image.BILINEAR)

    return (img, kps)

In [ ]:
# COPY & PASTE from second.ipynb
# Generic Imports
from scipy.spatial import Delaunay

def delaunay_graph(kpts: np.array):
    points = list(zip(kpts[0], kpts[1]))
    tri = Delaunay(points)
    edges = []

    for simplex in tri.simplices:
        for i in range(len(simplex)):
            for j in range(i + 1, len(simplex)):
                edges.append((simplex[i], simplex[j]))

    return list(set(tuple(sorted(e)) for e in edges))

In [ ]:
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment

def simple_spatial_matching(kpts1, kpts2):
    points1 = np.array(zip(kpts1[0], kpts1[1]))
    points2 = np.array(zip(kpts2[0], kpts2[1]))
    
    # Generate Matrixes Containing Euclidean distances
    cost_matrix = distance_matrix(points1, points2)
    # Use the Hungarian Algorithm
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    # Generate Matrix with Zeros
    matching_matrix = np.zeros(cost_matrix.shape, dtype=int)
    # Populate the Matching Matrix
    matching_matrix[row_ind, col_ind] = 1
    
    return matching_matrix

In [ ]:
def visualize_matching_full(triplet1, triplet2, matching_matrix):
    # Hace un plot
    img1, kpts1, edges1 = triplet1
    img2, kpts2, edges2 = triplet2
    # Prepare Canvas
    combined_img = np.hstack((img1, img2))
    # Create offset
    w1 = img1.shape[1]
    
    plt.figure(figsize=(12, 6))
    plt.imshow(combined_img)
    plt.axis('off')
    
    # Show Delaunay
    # 2. Draw Delaunay Structure (Yellow lines)
    # Graph 1
    for i, j in edges1:
        plt.plot([kpts1[0, i], kpts1[0, j]], [kpts1[1, i], kpts1[1, j]], 'y-', alpha=0.5, lw=1)
    # Graph 2 (Shifted x by w1)
    for i, j in edges2:
        plt.plot([kpts2[0, i] + w1, kpts2[0, j] + w1], [kpts2[1, i], kpts2[1, j]], 'y-', alpha=0.5, lw=1)
        
    # Matching Lines
    rows, cols = np.where(matching_matrix == 1)
    
    for i, j in zip(rows, cols):
        plt.plot([kpts1[0, i], kpts2[0, j] + w1], [kpts1[1, i], kpts2[1, j]], 'g-', lw=1.5, alpha=0.8)
        
    plt.scatter(kpt1[0], kpt1[1], c='w', edgecolors='k', s=40, zorder=5)
    plt.scatter(kpt2[0] + w1, kpt2[1], c='w', edgecolors='k', s=40, zorder=5)
    
    plt.title("Graph Matching Results")
    
    # Save the file instead of showing it
    plt.savefig('matching_result.jpg', bbox_inches='tight', dpi=300)
    plt.close() # Important: closes the plot to free up memory
    

In [ ]:
# Actual main code
def show_matching(cat: Categories, n: int = 2):
    pairs = getting(cat, n) # Gets
    pairs = [resizing(p) for p in pairs] # Resizes
    triplets = [(pair[0], pair[1], delaunay_graph(pair[1])) for pair in pairs]
    visualize_matching()


In [ ]:
# Execution Example
def show_delaunay(cat: Categories, n: int = 4):
    pairs = getting(cat, n) # Gets
    pairs = [resizing(p) for p in pairs] # Resizes
    triplets = [(pair[0], pair[1], delaunay_graph(pair[1])) for pair in pairs]
    visualize(triplets) # And plots

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def HungarianAlgorithm(cost_matrix):
    n = cost_matrix.shape[0]
    
    # Step 1 & 2: Row and Column reduction 
    cost_matrix = cost_matrix - cost_matrix.min(axis=1)[:, np.newaxis]
    cost_matrix = cost_matrix - cost_matrix.min(axis=0)

    while True:
        # Step 3: Find optimal assignment/cover zeros 
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        
        # If we found n independent zeros, we are done 
        if len(row_ind) == n:
            return row_ind, col_ind
        
        # Step 4: Create additional zeros 
        # First, identify covered rows and columns (simplified for this manual step)
        # In a full manual implementation, you'd calculate 'marked' lines.
        # Assuming you need to find the min value not covered by the current assignment:
        mask = np.ones(cost_matrix.shape, dtype=bool)
        mask[row_ind, :] = False
        mask[:, col_ind] = False
        
        min_uncovered = cost_matrix[mask].min() [cite: 9, 11]

        # Subtract from uncovered elements 
        cost_matrix[mask] -= min_uncovered
        
        # Add to elements covered by two lines (intersections) 
        # Elements at (row_ind, col_ind) are covered twice in the logic of the algorithm
        for r in row_ind:
            for c in col_ind:
                cost_matrix[r, c] += min_uncovered

In [ ]:
def find_minimum_uncovered_value():
    ...